# Week 4 - Supervised Learning: Evaluation and diagnostics

Example with Planet habitability

<hr style="border:2px solid gray">

## Index: <a id='index'></a>
1. [Research-level datasets](#dataset)
2. [Model evaluation](#evaluation)
3. [Cross validation](#crossvalidation)
4. [Learning curves](#learningcurves)


<hr style="border:2px solid gray">

## Research-level datasets: [^](#index) <a id='index'></a>

Last week we took our first steps in Machine Learning using a small dataset, this week we are getting our hands dirty with a research-level dataset, which will be much larger, messier and noisier than what we saw last week.

Let's remind ourselves of the steps of a machine learning project.
1. **Problem formulation:** Define the problem that you want to solve. What
are you trying to predict or classify? What data do you have available?
2. **Data collection:** Collect the data that you need to train the model.
This data should be representative of the data that you will use to make
predictions.
3. **Data preparation and feature engineering:** Prepare the data for training, such as cleaning and
transforming it. This may involve removing outliers, imputing missing
values, and normalizing the data. Select the features that are most important for the
problem. This may involve creating new features or removing irrelevant
features.
5. **Model selection and training:** Choose the machine learning algorithm that is most
suitable for your problem. There are many different machine learning
algorithms available, and the best algorithm for your problem will depend
on the specific data and the desired outcome. Train the model on the data. This involves feeding the
data to the algorithm and letting it learn how to make predictions.
7. **Model evaluation:** Evaluate the model on a held-out dataset. This is
a dataset that was not used to train the model. The evaluation will help
you to assess the accuracy of the model and identify any problems.
8. **Model tuning:** Tune the hyperparameters of the model to improve its
performance.

### STEP 1: Problem formulation 
Just like last week I will give you the first two steps, but it is a good idea anyway to write out your research question.

In the cell below, **define the problem that we want to solve**. 


 - #### We aim to build a supervised classifier that predicts whether an exoplanet is habitable.
 - #### Data available consists of a research‑scale exoplanet catalog with many columns.
 - #### Four predictive features: S_MASS (stellar mass), P_DISTANCE (planet mean orbital distance in AU), P_PERIOD (orbital period in days), and P_FLUX (mean stellar flux in Earth units)
 - #### one target P_HABITABLE

Before we start you will want to run all of these module imports!

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.pipeline import Pipeline


import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import colors

### STEP 2: Data collection
In the second step, we collect the data that we need to train our model. In this case we use again the [Habitable Worlds Catalogue]('https://phl.upr.edu/hwc'). It lists up to potentially habitable worlds in a list of over five thousand known exoplanets, putting together information gathered by several observatories, including the Kepler and K2 missions and the ongoing Transiting Exoplanet Survey Satellite.

Just like you did last week, use the `read_csv` function from the `panda` module to read in the `hwc.csv` file into a panda DataFrame.

In [3]:
pd_exo=pd.read_csv(r'hwc.csv')

Take a Quick Look at the Data Structure with `.head()`, `describe()`, and `info()`

In [4]:
pd_exo.head()

,P_NAME,P_MASS,P_MASS_ERROR_MIN,P_MASS_ERROR_MAX,P_RADIUS,P_RADIUS_ERROR_MIN,P_RADIUS_ERROR_MAX,P_YEAR,P_UPDATED,P_PERIOD,...,S_ABIO_ZONE,S_TIDAL_LOCK,P_HABZONE_OPT,P_HABZONE_CON,P_TYPE_TEMP,P_HABITABLE,P_ESI,S_CONSTELLATION,S_CONSTELLATION_ABR,S_CONSTELLATION_ENG
0,OGLE-2016-BLG-1227L b,251.084120,-123.952920,413.176400,13.90040,0.00000,0.00000,2020,2020,0.000000,...,0.000046,0.000000,0,0,Cold,0,0.146639,Scorpius,Sco,Scorpion
1,Kepler-276 c,16.527056,-3.496108,4.449592,2.90339,-0.28025,1.26673,2013,2014,31.884000,...,2.097783,0.316980,0,0,Hot,0,0.271883,Cygnus,Cyg,Swan
2,Kepler-829 b,5.085248,0.000000,0.000000,2.10748,-0.17936,0.43719,2016,2016,6.883376,...,1.756317,0.459559,0,0,Hot,0,0.254888,Lyra,Lyr,Lyre
3,K2-283 b,12.172812,0.000000,0.000000,3.51994,-0.15694,0.15694,2018,2018,1.921036,...,0.568374,0.000000,0,0,Hot,0,0.193908,Pisces,Psc,Fishes
4,Kepler-477 b,4.926334,0.000000,0.000000,2.07385,-0.12331,0.17936,2016,2016,11.119907,...,0.768502,0.386150,0,0,Hot,0,0.276524,Lyra,Lyr,Lyre


In [5]:
pd_exo.describe()

/home/codespace/.local/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/codespace/.local/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/codespace/.local/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/codespace/.local/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/codespace/.local/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/codespace/.local/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid v

,P_MASS,P_MASS_ERROR_MIN,P_MASS_ERROR_MAX,P_RADIUS,P_RADIUS_ERROR_MIN,P_RADIUS_ERROR_MAX,P_YEAR,P_UPDATED,P_PERIOD,P_PERIOD_ERROR_MIN,...,S_HZ_CON0_MAX,S_HZ_CON1_MIN,S_HZ_CON1_MAX,S_SNOW_LINE,S_ABIO_ZONE,S_TIDAL_LOCK,P_HABZONE_OPT,P_HABZONE_CON,P_HABITABLE,P_ESI
count,5429.000000,5429.000000,5429.000000,5566.000000,5566.000000,5566.000000,5566.000000,5566.000000,5.566000e+03,5.566000e+03,...,5566.000000,5566.000000,5566.000000,5566.000000,5.566000e+03,5566.000000,5566.000000,5566.000000,5566.000000,5263.000000
mean,435.512073,-56.044664,74.822279,5.673366,-0.291506,0.367247,2016.143909,2016.332734,7.684540e+04,-1.964066e+04,...,2.297740,1.232818,2.297740,3.567121,9.798415e+34,0.336715,0.060187,0.043478,0.025332,0.257500
std,2371.324125,243.280220,367.353833,5.313856,0.816212,1.320016,4.504200,4.446826,5.390851e+06,1.342383e+06,...,4.503969,2.346731,4.503969,6.620466,5.168606e+36,0.189695,0.237854,0.203949,0.210859,0.134995
min,0.000000,-4767.420000,0.000000,0.000000,-32.509000,0.000000,1992.000000,1992.000000,0.000000e+00,-1.000000e+08,...,0.001910,0.000911,0.001910,0.002434,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000019
25%,3.941067,-14.937916,0.000000,1.771180,-0.325090,0.000000,2014.000000,2014.000000,4.016440e+00,-9.675000e-04,...,1.084775,0.581461,1.084775,1.650019,4.140582e-01,0.278981,0.000000,0.000000,0.000000,0.178469
50%,8.498721,0.000000,0.000000,2.768870,-0.123310,0.134520,2016.000000,2016.000000,1.059206e+01,-5.780500e-05,...,1.624331,0.888340,1.624331,2.605336,1.282764e+00,0.432534,0.000000,0.000000,0.000000,0.267601
75%,155.735720,0.000000,15.891400,11.770500,0.000000,0.414770,2020.000000,2020.000000,3.837136e+01,-5.245000e-06,...,2.328600,1.278994,2.328600,3.813851,2.504773e+00,0.463091,0.000000,0.000000,0.000000,0.302495
max,89627.496000,0.000000,7945.700000,77.349000,0.000000,68.919080,2023.000000,2023.000000,4.020000e+08,0.000000e+00,...,120.348830,67.331558,120.348830,214.468620,2.726899e+38,1.003328,1.000000,1.000000,2.000000,0.950567


In [6]:
pd_exo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5566 entries, 0 to 5565
Columns: 103 entries, P_NAME to S_CONSTELLATION_ENG
dtypes: float64(85), int64(5), object(13)
memory usage: 4.4+ MB


This dataset has a lot of features! For the purpose of this exercise, it is useful to just focus on the following interesting features:

S_MASS - star mass (solar units)

P_DISTANCE - planet mean distance from the star (AU)

P_PERIOD - planet period (days)

P_FLUX - planet mean stellar flux (earth units)

Define a new dataset with just these three features and the labels `P_HABITABLE` and verify using `head()` or `describe()` that it worked.

In [7]:
pd_exo_int = pd_exo[["S_MASS", "P_DISTANCE", "P_PERIOD","P_FLUX","P_HABITABLE"]]

### STEP 3 Data preparation and Feature Engineering

Let's check the `P_HABITABLE` entries. You can use the `groupby` function of `panda` to group the dataframe by the `P_HABITABLE` feature. If you are not sure how to use it, check it out with `help` first.

In [8]:
pd_exo_int.groupby('P_HABITABLE').size()

P_HABITABLE
0    5480
1      31
2      55
dtype: int64

The three numbers mean:
```
0 --> not habitable
1 --> conservative
2 --> optimistic
```
It would be more useful for us just to have a binary option, so we should transform our dataset accordingly.

Let's start with creating a new dataset without the `P_HABITABLE` feature (hint: use `drop` from `panda`):

In [10]:
pd_exo_bin = pd_exo_int.drop("P_HABITABLE", axis = "columns")

Now let's add a column called `P_HABITABLE` to our new data frame, with:

```
0 --> not habitable
1 --> habitable
```

Now let's add a feature to our dataframe that is the `logical or` of the 1 and 2 `P_HABITABLE` original values (hint: use `logical_or` from `numpy`) 

NB `logical_or` will return True and False values, so you will also need to convert the column to integers. 
Once you are confident that you have performed the two operations correctly, check that `pd_exo_bin.head()` gives you the result you expect.

In [11]:
pd_exo_bin["P_HABITABLE"] = np.logical_or(
    pd_exo_int["P_HABITABLE"] == 1,
    pd_exo_int["P_HABITABLE"] == 2
).astype(int)

print(pd_exo_bin.tail())
print(pd_exo_bin["P_HABITABLE"].value_counts())

      S_MASS  P_DISTANCE     P_PERIOD      P_FLUX  P_HABITABLE
5561    1.02    0.053300     4.458149  348.775190            0
5562    0.39    2.440000     0.000000    0.167966            0
5563    0.84    0.000000     3.770150         inf            0
5564    0.53    1.900000     0.000000    0.277008            0
5565    1.21    5.205792  3999.000000    0.123796            0
P_HABITABLE
0    5480
1      86
Name: count, dtype: int64


Using the `numpy` functions `isnan` and `isinf`, check how many instances of NaN (not a number) and infinity are in the dataset.

In [12]:
### Counting missing data...
countInf = np.isnan(pd_exo_bin).sum()
countNaN = np.isinf(pd_exo_bin).sum()
print("There are {} instances of NaN in the dataset".format(countNaN))
print("There are {} instances of Infinity in the dataset".format(countInf))

There are S_MASS           0
P_DISTANCE       0
P_PERIOD         0
P_FLUX         303
P_HABITABLE      0
dtype: int64 instances of NaN in the dataset
There are S_MASS         0
P_DISTANCE     0
P_PERIOD       0
P_FLUX         0
P_HABITABLE    0
dtype: int64 instances of Infinity in the dataset


As we saw during the lecture, there are different ways of dealing with missing values. In our case we will choose a shortcut and replace all infinities with NaN, and then drop all of the instances of NaN in the dataset.

In [13]:
pd_exo_bin.replace([np.inf, -np.inf], np.nan, inplace=True)
pd_exo_bin.dropna(inplace=True)
final_features = pd_exo_bin
final_features.shape

(5263, 5)

The output from `final_features.shape` should give you `(5263, 5)`.

Now it's time to also define our targets as the `P_HABITABLE` column in the dataframe, and drop the `P_HABITABLE` feature from our dataframe.

In [14]:
targets = pd_exo_bin["P_HABITABLE"]
targets.head()

0    0
1    0
2    0
3    0
4    0
Name: P_HABITABLE, dtype: int64

In [15]:
final_features = pd_exo_bin.drop("P_HABITABLE", axis = "columns")
final_features.head()

,S_MASS,P_DISTANCE,P_PERIOD,P_FLUX
0,0.10,3.4000,0.000000,0.086505
1,1.10,0.1994,31.884000,20.490365
2,0.98,0.0678,6.883376,238.528680
3,0.89,0.0291,1.921036,353.357260
4,0.87,0.0911,11.119907,51.163853


**Number 1 rule of science: know your data!** 
Plot 4 histograms with the final features we chose. Use `describe` to inspect the dataset. Are there any outliers?

It is clear that are some big outliers in this dataset that may end up biasing our result. One way to deal with them is to remove all outliers that are more than 5 standard deviations away from the mean.

The `stats` `zscore` method gives you a way to evaluate how many sigmas away is a value from the mean. Use it to filter out instances that have features that are more than 5 sigmas away. Then verify using `describe` that the datasets have less outlieers

In [ ]:
final_features = ...


We will need to redefine our targets using the new indices. Run the cell below.

In [ ]:
targets = targets[final_features.index]

Reset the index of both final_features and targets

In [ ]:
final_features = final_features.reset_index(drop=True)
targets = targets.reset_index(drop=True)

<div style="background-color:#C2F5DD">

## Exercise 1
1. Size: What was the size of the data set before and after the cleaning up procedure?
2. Missing data: how did you deal with them? what could have been a better way of dealing with them?
3. Outliers: how did we deal with them?
4. Balance: check how many habitable planets are in the dataset and if it is a similar number to the non-habitable ones
5. Intuition: which model is it going to work best, decision trees or kNN? why?
6. Do the habitable and non-habitable planets have similar statical features? (hint: use `concat` to put together the `final_features` and `targets` dataframes, and `describe` using the `percintiles=[]` option and `groupby` `P_HABITABLE`)

### STEP 4: Model selection and training

Use the cell below to define the train and test sets, we use `random_state=2` for reproducibility.

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(final_features,targets,random_state=2)

In [ ]:
Xtrain.shape, Xtest.shape

<hr style="border:2px solid gray">

## STEP 5: Model evaluation [^](#index) <a id='evaluation'></a>

<div style="background-color:#C2F5DD">

## Exercise 2
1. Plot a scatter graph of the train and test set like you did last week.
2. Train a `DecisionTreeClassifier` with `random_state=3`.
3. Display the graph of the decision tree and count how many nodes it has. (hint: you can use `node_count` from the tree option)
4. Calculate the `accuracy_score`, `precision_score`, and `recall_score` for the train and test set. Write a comment on the performance of the classifier.
5. Compare the test set metrics to those you would find with 3 dummy classifiers: one that assigns everything to the habitable class, one that assigns everything to the non-habitable class, and one that is alternating 0s and 1s.
6. Display the 4 confusion matrices from the 3 dummy classifier and our decision tree one.
7. Which of the classifier performs better? Which has the best accuracy? best precision? best recall? Why? Justify each answer.
8. Plot the ROC curve. Comment on the result

<hr style="border:2px solid gray">

## Cross validation [^](#index) <a id='crossvalidation'></a>

The idea of cross validation is that we choose several possible train/test splits and repeat the training process accordingly. The overall performance can be estimated as the mean (or median) of the test scores obtained in all the attempts. The standard deviation (or other dispersion measures) provides an estimate of the uncertainty.

*k-fold* cross-validation is the most common strategy. It consists of dividing the learning set in *k* folds and cycling through the folds so that at each iteration one is the test set and the remaining *k-1* folds are the training set.

The 3 cells below show 3 types of Cross Validation:
 - the *standard version*: it doesn't shuffle the data, so if your positive examples are all at the beginning or all the end, it might lead to disastrous results.
 - the *shuffle version*: it shuffles the data as well 
 - the *stratification version*: it ensures that the class distributions in each split resembles those of the  entire data set


In [ ]:
cv1 = KFold(n_splits = 5) # standard

cv2 = KFold(shuffle = True, n_splits = 5, random_state=5) # shuffled

cv3 = StratifiedKFold(shuffle = True, n_splits = 5, random_state=5) # stratification

Let's look at the class count in each set of splits for the first cross validation case. Run the cell below.

In [ ]:
for train, test in cv1.split(final_features, targets): 
    print('train -  {}   |   test -  {}'.format(np.bincount(targets.loc[train]), np.bincount(targets.loc[test])))

The handy function *cross\_validate*  provides the scores (specified by the chosen scoring parameter), in dictionary form. I'll do it for the first cross validation and for *accuracy* as the chosen metrics:

In [ ]:
scores1 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv1, scoring = 'accuracy')
scores1

Calculate the mean and the standard deviation of the `test_score` entry in the score dictionaries.

 If desired, I can ask for the train scores as well and compare them to the test scores. This is very helpful when diagnosing bias vs variance.

In [ ]:
scores1 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv1, scoring = 'accuracy', \
                         return_train_score = True)
print("Accuracy train score: {:.2} +/- {:.2}".format(scores1['train_score'].mean(), scores1['train_score'].std()))
print("Accuracy test score: {:.2} +/- {:.2}".format(scores1['test_score'].mean(), scores1['test_score'].std()))

The cross\_validate function is useful to calculate the score, but does not produce predicted labels.

These can be obtained by using the `cross_val_predict` function, which saves the predictions for each of the k test folds, and compiles them together.

In [ ]:
model1 = DecisionTreeClassifier(random_state=3)
scores1 = cross_val_score(model1, final_features, targets, cv = cv1, scoring = 'accuracy')
y1 = cross_val_predict(model1, final_features, targets, cv = cv1)

In [ ]:
y1

A quick trick to see how many planets are predicted to be habitable (predicted label = 1) is just to use sum as below:

In [ ]:
np.sum(y1) 

This allows us to get a confusion matrix too

In [ ]:
metrics.confusion_matrix(targets,y1)

cm = metrics.confusion_matrix(targets,y1, labels=model.classes_)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=['Not Habitable','Habitable'])
disp.plot()

print("Number of True Negatives: {:.3f}".format(cm[0,0]))
print("Number of True Positives: {:.3f}".format(cm[1,1]))
print("Number of False Negatives: {:.3f}".format(cm[1,0]))
print("Number of False Positives: {:.3f}".format(cm[0,1]))

<div style="background-color:#C2F5DD">

## Exercise 3
1. Look at the class count for the other two types of cross validation, and compare all 3 class counts. Write a comment with your thoughts about them.
2. Calculate the scores for the three cross validations using `cross_validate` and find the their mean and standard deviation. Comment on the model performance.
3. Now repeat the same as above, but using `recall` as a metric, and then again using `precision` as a metric. Compare the results.
4. Compare the cross validation `recall` scores obtained from the train set and from the test set. Comment on the results.
5. Calculate the predicted labels for the stratified cross validation case and all 3 metrics (`accuracy`, `precision`, and `recall`) and compare the number of habitable planets found in each case and the confusion matrices.
6. Optional: plot the ROC curve

<div style="background-color:#C2F5DD">

## Exercise 4
1. Train a `kNearestNeighbour` classifier with `n_neighbors=3`.
2. Calculate the `accuracy_score`, `precision_score`, and `recall_score` for the train and test set, and compare them to those from the initial Decision Tree. Write a comment on the performance of the classifier.
3. Now scale the dataset using `RobustScaler` and retrain the classifier. How does it perform?
4. Define a pipeline using `pipeline = Pipeline([('transformer', scaler), ('estimator', model)])` and use stratified cross validation to evaluate the performance of the model. *Hint: once you have defined a pipeline you can use it in the `cross_validate` functions instead of the `model` input*
5. Hyperparameters tuning and optimisation: Optimise the hyperparameter $k$
6. Comment on the results, and identify the possible issues with the training.
7. 

<hr style="border:2px solid gray">

## Learning curves and hyperparameters tuning [^](#index)<a id='learningcurves'></a>
Learning curves are a useful diagnostic tool for supervised models. They are used to estimate how the performance of the model is tied to the size of the learning set. 
Run the cell below to see an example in which we find the train and test scores for our pipeline when using a sample of increasing size.

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(estimator = pipeline, X = final_features, y = targets, 
    cv = 5, train_sizes=np.linspace(0.1, 1.0, 5), scoring = 'recall')


<div style="background-color:#C2F5DD">

## Exercise 5
1. Calculate the mean and standard deviation of the train and test scores, and plot them as a function of the train size. You should use a solid line for the mean recall, and a shaded fill area (*hint `fill_between` with a suitable `alpha` option) for marking the error.
2. Now repeat the same methods and plot but using a model `DecisionTreeClassifier` like we defined earlier today. Make sure to include a plot title to easily distinguish your figures. Comment on the results.
3. It looks like our models are not performing great. Sometimes tuning hyper-parameters helps. Repeat the learning curves but for a `kNN` model with 5 and 7 neighbours. Comment on the results.
4. The decision tree we used at the beginning of the lab book had more than 100 nodes, can you get better performance by setting a maximum depth of the tree (*hint using `max_depth`). Use the learning curve to determine how the performance changes with the number of nodes. Comment on the behaviour.

Key points: plot Recall vs size of training dataset
To find if this is overfitting or underfitting (i.e check by using the training and test scores)
Seek optimal fitting